# Get The Data

To retrieve the data from <i>kaggle via </i>kaggle api, the <i>kaggle credentials</i> file should been in colab's virtual machine. In this project, the credentials file was in the drive. Therefore, the drive should be mounted to the virtual machine.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Than, copy the credentials file into the <b>.kaggle</b> folder where in the <b>root</b> folder. (Firstly you have to create that <b>.kaggle</b> folder in the <b>root</b> folder)

In [2]:
!mkdir /root/.kaggle
!cp drive/MyDrive/kaggle/kaggle.json /root/.kaggle/

In [3]:
!kaggle competitions download -c digit-recognizer  # see the kaggle api documentation

 55% 5.00M/9.16M [00:00<00:00, 10.3MB/s]
100% 9.16M/9.16M [00:00<00:00, 17.0MB/s]
 82% 5.00M/6.09M [00:00<00:00, 16.6MB/s]
100% 6.09M/6.09M [00:00<00:00, 17.5MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 66.2MB/s]


As you might expect, the <i>.csv</i> files are extracting from the archives.

In [4]:
!unzip test.csv.zip 
!unzip train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


# Prepare The Data 

In [5]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [7]:
train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
train_label = train["label"].copy()  # putting labels apart from the features.
train_features = train.drop(labels="label", axis=1) # putting features apart from the labels

In [9]:
X_train, X_valid = train_features[5000:], train_features[:5000]  # creating a validation set
y_train, y_valid = train_label[5000:], train_label[:5000]        # that composed of 5000 instances

In [10]:
X_train = X_train / 255    # since values are between 0 and 255, they can be normalized by dividing 255
X_valid = X_valid / 255    # this dividing operation ensures that whole values will be between 0 and 1
X_test = test / 255        # as another option, you can use sklearn.preprocessing.StandardScaler class

In [11]:
X_train.shape, X_valid.shape, X_test.shape

((37000, 784), (5000, 784), (28000, 784))

This operation ends up with convert the sets from dataframe to numpy array and also reshape the sets to ("number of instances", 28, 28)

In [12]:
X_train = np.array(X_train).reshape([X_train.shape[0], 28, 28])
X_valid = np.array(X_valid).reshape([X_valid.shape[0], 28, 28])
X_test = np.array(X_test).reshape([X_test.shape[0], 28, 28])

# Build a CNN Architecture 

In [13]:
from tensorflow import keras

In [14]:
model = keras.models.Sequential()  # creating a sequential model
model.add(keras.layers.Input(shape=X_train.shape[1:])) # an input layer with (28, 28) 
model.add(keras.layers.Conv1D(filters=64, kernel_size=4, activation="relu", padding="same"))
model.add(keras.layers.Conv1D(filters=128, kernel_size=4, activation="relu", padding="same"))
model.add(keras.layers.MaxPooling1D(1))
model.add(keras.layers.Conv1D(filters=256, kernel_size=2, activation="relu", padding="same"))
model.add(keras.layers.Conv1D(filters=128, kernel_size=2, activation="relu", padding="same"))
model.add(keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu", padding="same"))
model.add(keras.layers.MaxPooling1D(1))
model.add(keras.layers.Flatten()) # to feed the dense layers, the inputs have to convert to flatten shape
model.add(keras.layers.Dense(units=32, activation="relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(units=10, activation="softmax")) # since there are 10 classes, the dense layer
                                                              # has 10 nuerons as output neurons 
                                                              # with softmax activation function

In [15]:
model.compile(optimizer="nadam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) # self-explained

In [16]:
model.fit(x=X_train, y=y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1157/1157 [==============================] - 20s 11ms/step - loss: 0.8782 - accuracy: 0.6977 - val_loss: 0.1108 - val_accuracy: 0.9690
Epoch 2/10
1157/1157 [==============================] - 11s 10ms/step - loss: 0.2736 - accuracy: 0.9160 - val_loss: 0.1027 - val_accuracy: 0.9748
Epoch 3/10
1157/1157 [==============================] - 11s 10ms/step - loss: 0.1971 - accuracy: 0.9374 - val_loss: 0.0659 - val_accuracy: 0.9812
Epoch 4/10
1157/1157 [==============================] - 11s 10ms/step - loss: 0.1652 - accuracy: 0.9447 - val_loss: 0.0520 - val_accuracy: 0.9862
Epoch 5/10
1157/1157 [==============================] - 11s 10ms/step - loss: 0.1502 - accuracy: 0.9494 - val_loss: 0.0720 - val_accuracy: 0.9816
Epoch 6/10
1157/1157 [==============================] - 11s 10ms/step - loss: 0.1376 - accuracy: 0.9533 - val_loss: 0.0811 - val_accuracy: 0.9822
Epoch 7/10
1157/1157 [==============================] - 11s 10ms/step - loss: 0.1278 - accuracy: 0.9589 - val_loss: 0.0602 -

# Prediction and Submitting

In [17]:
y_pred = model.predict(X_test)

In [19]:
predicted_classes = [prediction.argmax() for prediction in y_pred] # selecting the most possible class

In [24]:
# competition's submission procedures
submission = pd.DataFrame(predicted_classes, columns=["Label"])
submission.index.name = "ImageId"
submission.index += 1
submission.to_csv("submission.csv")